In [1]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm, poisson
from scipy.optimize import minimize

# Usando um parâmetro por jogador

In [2]:
def generate_games():
    games = [[chr(i + 65) for i in range(20)] for j in range(20)]
    for i in range(20):
        games[i].pop(i)
        
    return games

def generate_players(temps = 15):
    players = list(np.random.normal(70, 10, 220))
    players = [round(player, 1) if player < 100 else 100 for player in players]
    clubs = {}
    for year in range(temps):
        line_up = {}
        if type(players) != list:
            np.random.shuffle(list(players.flat))

        players = np.reshape(players, (20, 11))
        for club in range(20):
            line_up[chr(65 + club)] = players[club]

        clubs[year] = line_up
        
    return clubs

def find_median(array):
    arraycopy = array.copy()
    arraycopy.sort()
    if len(arraycopy) % 2 == 0:
        return (arraycopy[len(arraycopy) // 2 - 1] + arraycopy[len(arraycopy) // 2])/2
    else:
        return arraycopy[len(arraycopy) // 2]
    
def find_forces(clubs, year = 0):
    atk_forces = []
    def_forces = []
    for club in clubs[year]:
        atk_forces.append(np.sum(clubs[year][club]))
        def_forces.append(np.log(np.prod(clubs[year][club])))
            
    atk_med = find_median(atk_forces)
    def_med = find_median(def_forces)
    
    for i in range(len(atk_forces)):
        atk_forces[i] = atk_forces[i] / atk_med * 2
        def_forces[i] /= def_med
            
    return atk_forces, def_forces

def model(club1, club2, sims = 10000):
    '''
    receives 2 clubs (forces) and returns the most likely result, probability of each result and probability for the game
    '''
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(club1[0] - club2[1])
        goals2 = poisson.rvs(club2[0] - club1[1])
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [3]:
def championship(clubs, sims = 10000, year = 0):
    atk_forces, def_forces = find_forces(clubs, year = year)
    games = generate_games()
    results = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            club1 = (atk_forces[home], def_forces[home])
            club2 = (atk_forces[away], def_forces[away])
            _, result, _ = model(club1, club2, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results

In [4]:
clubs = generate_players(temps = 1)
atk_forces, def_forces = find_forces(clubs)

# exemplo
club1 = (atk_forces[ord('A') - 65], def_forces[ord('A') - 65])
club2 = (atk_forces[ord('B') - 65], def_forces[ord('B') - 65])
results, result, probs = model(club1, club2, sims = 10000)

print('Forças do clube 1:', club1)
print('Forças do clube 2:', club2)
print()
print('Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):')
print(results)
print()
print('O resultado mais provável é {} x {}.'.format(result[0], result[1]))
print('Além disso, a probabilidade do mandante é {}, enquanto a do visitante é de {} e a de empate é {}.'.format(probs[0], probs[2], probs[1]))

Forças do clube 1: (1.9693267481154144, 0.9956026324036424)
Forças do clube 2: (1.9649077203015337, 0.9945831775752193)

Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):
[[1471 1396  638  221   52   13]
 [1341 1336  687  218   61   10]
 [ 692  656  299  112   19    8]
 [ 246  210   90   30    9    1]
 [  58   65   24    7    2    0]
 [  13    7    5    0    2    1]]

O resultado mais provável é 0 x 0.
Além disso, a probabilidade do mandante é 0.3416, enquanto a do visitante é de 0.3445 e a de empate é 0.3139.


In [5]:
df, results = championship(clubs, sims = 100)
df

,points,games,wins,draws,defeats,goals for,goals against,goal difference
F,67,38,17,16,5,29,17,12
Q,64,38,15,19,4,27,16,11
E,62,38,16,14,8,24,15,9
I,61,38,15,16,7,28,20,8
C,59,38,15,14,9,24,18,6
N,59,38,13,20,5,29,21,8
D,57,38,13,18,7,21,14,7
L,52,38,11,19,8,18,15,3
J,51,38,9,24,5,19,15,4
G,48,38,10,18,10,17,17,0


In [6]:
results

[['A', 0, 0, 'B'],
 ['A', 0, 1, 'C'],
 ['A', 0, 1, 'D'],
 ['A', 1, 0, 'E'],
 ['A', 0, 1, 'F'],
 ['A', 0, 0, 'G'],
 ['A', 1, 1, 'H'],
 ['A', 1, 1, 'I'],
 ['A', 0, 0, 'J'],
 ['A', 0, 0, 'K'],
 ['A', 1, 1, 'L'],
 ['A', 0, 1, 'M'],
 ['A', 1, 1, 'N'],
 ['A', 0, 0, 'O'],
 ['A', 0, 0, 'P'],
 ['A', 0, 0, 'Q'],
 ['A', 1, 0, 'R'],
 ['A', 1, 1, 'S'],
 ['A', 0, 0, 'T'],
 ['B', 1, 1, 'A'],
 ['B', 0, 1, 'C'],
 ['B', 0, 1, 'D'],
 ['B', 1, 0, 'E'],
 ['B', 1, 0, 'F'],
 ['B', 0, 0, 'G'],
 ['B', 1, 0, 'H'],
 ['B', 0, 1, 'I'],
 ['B', 0, 0, 'J'],
 ['B', 0, 0, 'K'],
 ['B', 1, 0, 'L'],
 ['B', 0, 0, 'M'],
 ['B', 0, 1, 'N'],
 ['B', 1, 1, 'O'],
 ['B', 1, 1, 'P'],
 ['B', 0, 1, 'Q'],
 ['B', 0, 0, 'R'],
 ['B', 0, 0, 'S'],
 ['B', 1, 0, 'T'],
 ['C', 0, 0, 'A'],
 ['C', 1, 0, 'B'],
 ['C', 0, 0, 'D'],
 ['C', 1, 0, 'E'],
 ['C', 1, 0, 'F'],
 ['C', 1, 0, 'G'],
 ['C', 1, 1, 'H'],
 ['C', 1, 2, 'I'],
 ['C', 1, 0, 'J'],
 ['C', 1, 0, 'K'],
 ['C', 0, 1, 'L'],
 ['C', 1, 0, 'M'],
 ['C', 0, 1, 'N'],
 ['C', 0, 1, 'O'],
 ['C', 1, 1,

Perceba que fazendo

$força~de~ataque~do~time = \sum força~dos~jogadores \text{ e } força~de~defesa~do~time = \sum \log{força~dos~jogadores},$

um time com maior força de ataque terá maior força de defesa, o que não é compatível com a realidade.

# Usando dois parâmetros por jogador

Cada jogador tem dois parâmetros: ataque e defesa

In [7]:
def generate_players(clubs = 20, players_per_club = 23):
    '''
    creates players
    '''
    n_players = clubs * players_per_club
    players_atk = truncnorm.rvs(0, 10, loc = 2, scale = 0.5, size = n_players)
    players_def = truncnorm.rvs(0, 10, loc = 1, scale = 0.5, size = n_players)
    players = {}
    for i in range(n_players):
        players['P{:04d}'.format(i)] = [round(players_atk[i], 8) if players_atk[i] < 100 else 100,
                                        round(players_def[i], 8) if players_def[i] < 100 else 100]
        
    return players

def change_clubs(clubs, changes):
    '''
    change clubs
    '''
    clubs_f = {}
    changing = []
    change = 0
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        np.random.shuffle(club_players)
        for j in range(changes[i]):
            changing.append(club_players[j])

    np.random.shuffle(changing)
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        ind = []
        for player in club_players:
            if player in changing:
                ind.append(club_players.index(player))

        for j in range(len(ind)):
            club_players[ind[j]] = changing[change]
            change += 1

        clubs_f[chr(i + 65)] = club_players

    return clubs_f

def generate_clubs(players, years = 15, n_clubs = 20, players_per_club = 23):
    '''
    receives players and allocates them to clubs
    '''
    clubs = {}
    all_players = list(players)
    for year in range(years):
        line_up = {chr(65 + club) : {} for club in range(n_clubs)}
        clubs[year] = line_up
        for club in range(n_clubs):
            if year == 0:
                clubs[year][chr(65 + club)] = all_players[players_per_club * club:players_per_club * (club + 1)]

            else:
                changes = poisson.rvs(players_per_club/4, size = n_clubs)
                for i in range(n_clubs):
                    if changes[i] > players_per_club:
                        changes[i] = players_per_club

                    clubs[year] = change_clubs(clubs[year - 1], changes)
    return clubs

In [8]:
years = 15
n_clubs = 20
players_per_club = 23
players = generate_players()
clubs = generate_clubs(players)

In [9]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

def model(club1, club2, players, sims = 10000):
    '''
    receives 2 clubs (line_up) and returns the most likely result, probability of each result and probability for the game
    '''
    atk1, def1 = find_forces(club1, players)
    atk2, def2 = find_forces(club2, players)
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(atk1 - def2)
        goals2 = poisson.rvs(atk2 - def1)
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [10]:
def championship(clubs, players, sims = 10000, year = 0):
    games = generate_games()
    results = []
    line_up = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            
            club1 = clubs[year][chr(home + 65)].copy()
            np.random.shuffle(club1)
            home_line_up = club1[:11]
            
            club2 = clubs[year][chr(away + 65)].copy()
            np.random.shuffle(club2)
            away_line_up = club2[:11]
            
            _, result, _ = model(home_line_up, away_line_up, players, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            line_up.append([home_line_up, away_line_up])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results, line_up

In [11]:
table, results, line_up = championship(clubs, players, sims = 100, year = 0)

In [12]:
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
R,66,38,18,12,8,71,60,11
H,65,38,17,14,7,76,66,10
O,61,38,17,10,11,70,63,7
T,61,38,16,13,9,71,66,5
P,57,38,16,9,13,72,65,7
N,56,38,14,14,10,75,67,8
S,50,38,12,14,12,68,66,2
G,50,38,12,14,12,72,71,1
L,49,38,12,13,13,61,63,-2
C,49,38,11,16,11,66,68,-2


In [13]:
results

[['A', 1, 1, 'B'],
 ['A', 2, 0, 'C'],
 ['A', 3, 2, 'D'],
 ['A', 2, 2, 'E'],
 ['A', 2, 2, 'F'],
 ['A', 2, 2, 'G'],
 ['A', 2, 2, 'H'],
 ['A', 1, 2, 'I'],
 ['A', 1, 1, 'J'],
 ['A', 1, 2, 'K'],
 ['A', 1, 2, 'L'],
 ['A', 1, 1, 'M'],
 ['A', 2, 3, 'N'],
 ['A', 1, 2, 'O'],
 ['A', 2, 1, 'P'],
 ['A', 3, 1, 'Q'],
 ['A', 2, 3, 'R'],
 ['A', 1, 0, 'S'],
 ['A', 2, 2, 'T'],
 ['B', 1, 3, 'A'],
 ['B', 2, 2, 'C'],
 ['B', 2, 3, 'D'],
 ['B', 4, 2, 'E'],
 ['B', 1, 2, 'F'],
 ['B', 2, 1, 'G'],
 ['B', 2, 1, 'H'],
 ['B', 1, 1, 'I'],
 ['B', 1, 2, 'J'],
 ['B', 1, 0, 'K'],
 ['B', 1, 1, 'L'],
 ['B', 1, 1, 'M'],
 ['B', 2, 3, 'N'],
 ['B', 1, 2, 'O'],
 ['B', 2, 2, 'P'],
 ['B', 2, 1, 'Q'],
 ['B', 2, 1, 'R'],
 ['B', 2, 1, 'S'],
 ['B', 1, 1, 'T'],
 ['C', 2, 2, 'A'],
 ['C', 2, 2, 'B'],
 ['C', 1, 2, 'D'],
 ['C', 1, 1, 'E'],
 ['C', 2, 2, 'F'],
 ['C', 2, 3, 'G'],
 ['C', 1, 1, 'H'],
 ['C', 2, 2, 'I'],
 ['C', 2, 1, 'J'],
 ['C', 2, 1, 'K'],
 ['C', 2, 2, 'L'],
 ['C', 2, 1, 'M'],
 ['C', 2, 2, 'N'],
 ['C', 2, 1, 'O'],
 ['C', 1, 1,

In [14]:
line_up

[[['P0012',
   'P0010',
   'P0001',
   'P0015',
   'P0009',
   'P0004',
   'P0017',
   'P0006',
   'P0002',
   'P0018',
   'P0019'],
  ['P0032',
   'P0027',
   'P0039',
   'P0034',
   'P0023',
   'P0041',
   'P0038',
   'P0030',
   'P0043',
   'P0036',
   'P0042']],
 [['P0007',
   'P0002',
   'P0017',
   'P0012',
   'P0003',
   'P0021',
   'P0005',
   'P0014',
   'P0018',
   'P0016',
   'P0015'],
  ['P0063',
   'P0051',
   'P0058',
   'P0064',
   'P0068',
   'P0050',
   'P0047',
   'P0054',
   'P0053',
   'P0056',
   'P0065']],
 [['P0013',
   'P0022',
   'P0017',
   'P0006',
   'P0000',
   'P0007',
   'P0016',
   'P0002',
   'P0011',
   'P0012',
   'P0021'],
  ['P0084',
   'P0081',
   'P0069',
   'P0091',
   'P0085',
   'P0070',
   'P0088',
   'P0083',
   'P0089',
   'P0086',
   'P0090']],
 [['P0004',
   'P0018',
   'P0006',
   'P0007',
   'P0008',
   'P0016',
   'P0021',
   'P0010',
   'P0002',
   'P0022',
   'P0000'],
  ['P0092',
   'P0100',
   'P0114',
   'P0107',
   'P0094',
   'P0

# Engenharia reversa

In [15]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

# def find_players(players_er, codes):
#     players = {}
#     for code in codes:
#         if code not in players:
#             players[code] = players_er[codes.index(code)]
    
# #     for player in players_er:
# #         if player[0] not in players:
# #             players[player[0]] = players_er[players_er.index(player)][1:]
    
#     return players

def find_players(players_er, codes):
    players = {}
    for code in codes:
        if code not in players:
            players[code] = [players_er[2 * codes.index(code)], players_er[2 * codes.index(code) + 1]]
    
    return players

def likelihood(players_er, codes, results, line_up):
    '''
    players : parameter list
    results : list
    line_up : list
    '''
    players_er = find_players(players_er, codes)
    loglikelihood = 0
    for game in line_up:               
        ind = line_up.index(game)
        atk1, def1 = find_forces(game[0], players_er)
        atk2, def2 = find_forces(game[1], players_er)
        
        loglikelihood += poisson.logpmf(results[ind][1], atk1 / def2)
        loglikelihood += poisson.logpmf(results[ind][2], atk2 / def1)
        
    return -loglikelihood

In [16]:
players_er = {}
codes = []
for game in line_up:
    for player in game[0]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 3, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)
    for player in game[1]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 1, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)

# players_er = [players_er[player] for player in players_er]
aux = players_er.copy()
players_er = []
for i in aux:
    players_er.append(aux[i][0])
    players_er.append(aux[i][1])

lik = likelihood(players_er, codes, results, line_up)
lik

1073.7714378794788

In [17]:
result = minimize(likelihood, players_er, args = (codes, results, line_up), bounds = [(0, None) for player in players_er])

In [18]:
result

      fun: 949.7789501242237
 hess_inv: <920x920 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.26306077e-02,  5.50244298e-03, -1.88833839e-02, -4.19504434e-03,
        2.55454324e-02,  1.65528037e-02,  1.04591891e-02,  9.19726523e-03,
       -1.34264156e-02,  1.36185463e-01, -1.19371180e-03, -1.92244444e-02,
        3.13775674e-03,  1.20849108e-02,  3.13775674e-03,  1.15278453e-02,
       -8.94715418e-03, -1.33354661e-02, -1.57797318e-02, -1.10389919e-02,
       -1.16756383e-02,  5.42286219e-03, -2.27373677e-04, -1.77351468e-03,
       -1.45519153e-02, -5.26370062e-03, -1.90539139e-02,  1.04785158e-01,
        1.87583283e-03,  3.25826475e-02, -4.17230697e-03, -8.71978051e-03,
       -2.32716956e-02,  8.02629079e-03,  3.34694050e-02, -5.68434192e-03,
        1.11413102e-03,  4.29281499e-02, -2.48974175e-03,  6.62794268e-03,
        1.19257493e-02, -5.10453904e-03, -5.00222086e-04, -7.54880540e-03,
        2.09183783e-03, -1.11867849e-02, -9.19726523e-03,  6.99060365e-02,


In [19]:
result.nit

16

In [20]:
result.x

array([1.23074872e-01, 1.71272608e+00, 1.71405324e+00, 1.42128001e+00,
       7.03251557e-02, 4.00218153e+00, 4.75174415e+00, 3.73081729e+00,
       1.16359939e+00, 0.00000000e+00, 1.20066589e+00, 1.52127921e+00,
       2.65622622e+00, 5.07745493e-01, 3.36109894e+00, 5.54336406e-02,
       1.59849327e+00, 5.39782143e+00, 8.69638683e+00, 9.91541963e-02,
       3.08033994e+00, 1.44903155e+00, 1.42383123e+00, 3.34335683e+00,
       3.89792978e+00, 1.51988495e+00, 7.49043427e-01, 0.00000000e+00,
       1.25147782e+00, 7.39677277e-01, 1.05024184e+00, 3.60701657e+00,
       5.61981712e-01, 3.85825273e+00, 0.00000000e+00, 5.82113252e+00,
       5.76177932e+00, 0.00000000e+00, 3.62660833e-01, 1.56599235e+00,
       0.00000000e+00, 5.12291933e+00, 0.00000000e+00, 9.48344523e+00,
       3.10993292e+00, 5.67716808e+00, 5.68584808e+00, 0.00000000e+00,
       4.12444216e+00, 6.70758627e-01, 6.69985420e-02, 5.01496832e+00,
       0.00000000e+00, 4.52070188e+00, 2.38536019e+00, 1.23439748e+00,
      